# Appendix C, Example 3

## Introduction

In [ ]:
(refer 'sparqler.core)
(assert kabob)

## Query

### Manual SPARQL

The following cell will be stored in the beakerx.query1 variable....

In [ ]:
%%sql
# find six pairs of proteins that have been localized to the 
#  same cellular location and that are known to interact.
#
PREFIX franzOption_clauseReorderer: <franz:identity>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX kbio: <http://kabob.ucdenver.edu/bio/>

SELECT ?gp1 ?gp2 ?locationName
WHERE {

## get localization events
?loc1 rdfs:subClassOf obo:GO_0051179 . #localization

## get the entity localized
?loc1 rdfs:subClassOf ?gp1Loc .
?gp1Loc owl:onProperty obo:RO_0002313 . # RO:transports or maintains localization of .
?gp1Loc owl:someValuesFrom ?gp1_sc .
?gp1_sc rdfs:subClassOf ?gp1 .
?gp1_other_sc rdfs:subClassOf ?gp1 .

## get the location
?loc1 rdfs:subClassOf ?to .
?to owl:onProperty obo:RO_0002339 . # RO:has target end location
?to owl:someValuesFrom ?location_sc .

## name of the location
?location_sc rdfs:subClassOf ?location .
?location rdfs:label ?locationName .

## events the first entity participates in
?gp1Participant owl:someValuesFrom ?gp1_other_sc .
?gp1Participant owl:onProperty obo:RO_0000057 .
?interaction rdfs:subClassOf ?gp1Participant .

## look at only interactions
?interaction rdfs:subClassOf obo:MI_0407 . # interaction

## get all the other participants in those interactions
?interaction rdfs:subClassOf ?gp2Participant.
FILTER (?gp1Participant != ?gp2Participant) . # ensure different

## make sure one is the second entity
?gp2Participant owl:onProperty obo:RO_0000057 . # RO:has_participant
?gp2Participant owl:someValuesFrom ?gp2_sc .
?gp2_sc rdfs:subClassOf ?gp2 .
FILTER (?gp1 != ?gp2)

## get the potentially “co-localized” entity
?gp2_other_sc rdfs:subClassOf ?gp2 .
?gp2Loc owl:someValuesFrom ?gp2_other_sc .
?gp2Loc owl:onProperty obo:RO_0002313 . # RO:transports or maintains localization of .
?loc2 rdfs:subClassOf ?gp2Loc .
?loc2 rdfs:subClassOf obo:GO_0051179 . #localization
FILTER (?loc1 != ?loc2) . # ensure different

## find a second distinct localization to the same place
?loc2 rdfs:subClassOf ?to2 .
?to2 owl:onProperty obo:RO_0002339 . # RO:has target end location
?to2 owl:someValuesFrom ?location_sc2 .
?location_sc2 rdfs:subClassOf ?location .

}

In [ ]:
%%javascript
beakerx.query1 = beakerx.getCodeCells('query1')[0].code

### SPARQLer

#### Step 1: Convert to clojure and define constants

In [ ]:
(in-ns 'sparqler.core)

(defn all-gene-products [geneproduct gorgporv]
  (list geneproduct rdfs:subClassOf* gorgporv))

(defn interactions-geneproduct-participate-in [participant geneproduct interactions]
  (subquery [participant geneproduct interactions]
    participant owl:someValuesFrom geneproduct \.
    participant owl:onProperty RO:has_participant \. ;# RO:has_participant
    interactions rdfs:subClassOf participant \.))
            

In [ ]:
(in-ns 'sparqler.core)

(defquery KaBOBC3 []
 # find six pairs of proteins that have been localized to the 
#  same cellular location and that are known to interact.
#
PREFIX franzOption_clauseReorderer: <franz:identity>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX kbio: <http://kabob.ucdenver.edu/bio/>

SELECT ?gp1 ?gp2 ?locationName
WHERE {

## get localization events
?loc1 rdfs:subClassOf obo:GO_0051179 . #localization

## get the entity localized
?loc1 rdfs:subClassOf ?gp1Loc .
?gp1Loc owl:onProperty obo:RO_0002313 . # RO:transports or maintains localization of .
?gp1Loc owl:someValuesFrom ?gp1_sc .
?gp1_sc rdfs:subClassOf ?gp1 .
?gp1_other_sc rdfs:subClassOf ?gp1 .

## get the location
?loc1 rdfs:subClassOf ?to .
?to owl:onProperty obo:RO_0002339 . # RO:has target end location
?to owl:someValuesFrom ?location_sc .

## name of the location
?location_sc rdfs:subClassOf ?location .
?location rdfs:label ?locationName .

## events the first entity participates in
?gp1Participant owl:someValuesFrom ?gp1_other_sc .
?gp1Participant owl:onProperty obo:RO_0000057 .
?interaction rdfs:subClassOf ?gp1Participant .

## look at only interactions
?interaction rdfs:subClassOf obo:MI_0407 . # interaction

## get all the other participants in those interactions
?interaction rdfs:subClassOf ?gp2Participant.
FILTER (?gp1Participant != ?gp2Participant) . # ensure different

## make sure one is the second entity
?gp2Participant owl:onProperty obo:RO_0000057 . # RO:has_participant
?gp2Participant owl:someValuesFrom ?gp2_sc .
?gp2_sc rdfs:subClassOf ?gp2 .
FILTER (?gp1 != ?gp2)

## get the potentially “co-localized” entity
?gp2_other_sc rdfs:subClassOf ?gp2 .
?gp2Loc owl:someValuesFrom ?gp2_other_sc .
?gp2Loc owl:onProperty obo:RO_0002313 . # RO:transports or maintains localization of .
?loc2 rdfs:subClassOf ?gp2Loc .
?loc2 rdfs:subClassOf obo:GO_0051179 . #localization
FILTER (?loc1 != ?loc2) . # ensure different

## find a second distinct localization to the same place
?loc2 rdfs:subClassOf ?to2 .
?to2 owl:onProperty obo:RO_0002339 . # RO:has target end location
?to2 owl:someValuesFrom ?location_sc2 .
?location_sc2 rdfs:subClassOf ?location .

}
  )


In [ ]:
(#'sparqler.core/KaBOBC3)

In [ ]:
(beakerx :set "query2" (#'sparqler.core/KaBOBC3))

In [ ]:
%%html
<pre id="display"></pre>

In [ ]:
%%javascript
document.getElementById("display").innerHTML = "";

var one = beakerx.query1,
    other = beakerx.query2,
    color = 'blue',
    span = null;

var diff = JsDiff.diffWords(one, other, {ignoreWhitespace:true, newlineIsToken: true}),
    display = document.getElementById('display'),
    fragment = document.createDocumentFragment();

diff.forEach(function(part){
  // green for additions, red for deletions
  // grey for common parts
  color = part.added ? 'green' :
    part.removed ? 'red' : 'grey';
  span = document.createElement('span');
  span.style.color = color;
  span.appendChild(document
    .createTextNode(part.value));
  fragment.appendChild(span);
});

display.appendChild(fragment);

In [ ]:
(def q (#'sparqler.core/KaBOBC3))
q

In [ ]:
(def r (kabob-query q))
(clojure.set/rename r {:participant "participant"})

In [ ]:
(webview-query q)